# Solemne 1 - Algoritmos de Optimización

**Estudiante:** [Nombre del estudiante]  
**Fecha:** 29 de septiembre de 2024  
**Curso:** Algoritmos y Estructuras de Datos

Esta solemne presenta la implementación de cuatro desafíos algorítmicos utilizando diferentes técnicas de resolución:
1. Cálculo de determinantes (recursivo vs iterativo)
2. Problema de la mochila (backtracking vs fuerza bruta)
3. Asignación óptima de memoria (backtracking, DFS, fuerza bruta)
4. Resolución de Sudoku (backtracking)

---

In [ ]:
# Librerías necesarias
import numpy as np
import random

## Desafío 1: Resolución de Determinantes

**Objetivo:** Calcular el determinante de una matriz cuadrada utilizando dos enfoques diferentes:
- **Algoritmo recursivo:** Expansión por cofactores
- **Algoritmo iterativo:** Eliminación gaussiana

**Consideraciones especiales:**
- Matrices de tamaño 0x0 tienen determinante = 1 por convención
- El caso base 1x1 se maneja directamente
- Se generan matrices aleatorias con elementos entre -10 y 10

**Complejidad temporal:**
- Recursivo: O(n!)
- Iterativo: O(n³)

In [ ]:
# Solicitar tamaño de matriz
n = 0
while n <= 0:
    n = int(input("Ingresa el tamaño de la matriz (n x n): "))
    if n <= 0:
        print("Error: El tamaño debe ser un número positivo mayor que 0")

In [ ]:
# Generar matriz aleatoria
matriz = [[random.randint(-10, 10) for _ in range(n)] for _ in range(n)]

print("Matriz generada:")
for fila in matriz:
    print(fila)

In [ ]:
# Método recursivo - Expansión por cofactores
def determinante_recursivo(matriz):
    n = len(matriz)
    
    # Caso base
    if n == 1:
        return matriz[0][0]
    
    det = 0
    for j in range(n):
        # Crear submatriz eliminando fila 0 y columna j
        submatriz = [[matriz[i][k] for k in range(n) if k != j] 
                     for i in range(1, n)]
        
        # Calcular cofactor con signo alternado
        signo = (-1) ** j
        cofactor = signo * matriz[0][j] * determinante_recursivo(submatriz)
        det += cofactor
    
    return det

det_recursivo = determinante_recursivo(matriz)
print(f"Determinante (método recursivo): {det_recursivo}")

In [ ]:
# Método iterativo - Eliminación gaussiana
def determinante_iterativo(matriz):
    # Copia para no modificar la original
    mat = [fila[:] for fila in matriz]
    n = len(mat)
    det = 1
    
    for i in range(n):
        # Buscar pivote máximo
        pivote_fila = max(range(i, n), key=lambda j: abs(mat[j][i]))
        
        # Si el pivote es cero, determinante = 0
        if abs(mat[pivote_fila][i]) < 1e-10:
            return 0
        
        # Intercambiar filas si es necesario
        if pivote_fila != i:
            mat[i], mat[pivote_fila] = mat[pivote_fila], mat[i]
            det *= -1
        
        pivote = mat[i][i]
        det *= pivote
        
        # Eliminar elementos debajo del pivote
        for j in range(i + 1, n):
            factor = mat[j][i] / pivote
            for k in range(i + 1, n):
                mat[j][k] -= factor * mat[i][k]
    
    return det

det_iterativo = determinante_iterativo(matriz)
print(f"Determinante (método iterativo): {det_iterativo}")

## Desafío 2: Problema de la Mochila

**Objetivo:** Resolver el problema clásico de la mochila 0/1 donde se debe maximizar el valor de los objetos seleccionados sin exceder la capacidad de peso.

**Enfoques implementados:**
- **Backtracking:** Exploración sistemática con poda
- **Fuerza bruta:** Evaluación de todas las combinaciones posibles

**Formulación del problema:**
- Maximizar: Σ(valor_i × x_i)
- Sujeto a: Σ(peso_i × x_i) ≤ capacidad
- Donde x_i ∈ {0, 1}

**Complejidad:** O(2^n) para ambos métodos en el peor caso

In [ ]:
# Configuración del problema
n_objetos = int(input("Ingrese el número de objetos: "))
capacidad = int(input("Ingrese la capacidad máxima de la mochila: "))

# Generar objetos aleatorios (nombre, peso, valor)
objetos = [(f"Objeto_{i+1}", random.randint(1, capacidad), random.randint(1, 10)) 
           for i in range(n_objetos)]

print("\nObjetos disponibles:")
for nombre, peso, valor in objetos:
    print(f"{nombre}: peso={peso}, valor={valor}")

In [ ]:
# Método backtracking
def mochila_backtracking(objetos, capacidad):
    mejor_valor, mejor_combinacion = 0, []
    
    def bt(i=0, peso=0, valor=0, comb=[]):
        nonlocal mejor_valor, mejor_combinacion
        
        if i == len(objetos):
            if valor > mejor_valor:
                mejor_valor, mejor_combinacion = valor, comb.copy()
            return
        
        nombre, p, v = objetos[i]
        
        # No tomar el objeto
        bt(i + 1, peso, valor, comb)
        
        # Tomar el objeto si cabe
        if peso + p <= capacidad:
            comb.append(nombre)
            bt(i + 1, peso + p, valor + v, comb)
            comb.pop()
    
    bt()
    peso_total = sum(p for n, p, v in objetos if n in mejor_combinacion)
    return mejor_valor, mejor_combinacion, peso_total

valor_bt, combinacion_bt, peso_bt = mochila_backtracking(objetos, capacidad)
print(f"\nBacktracking - Valor: {valor_bt}, Objetos: {combinacion_bt}, Peso: {peso_bt}/{capacidad}")

In [ ]:
# Método fuerza bruta
def mochila_fuerza_bruta(objetos, capacidad):
    n = len(objetos)
    mejor_valor, mejor_combinacion = 0, []
    
    # Probar todas las 2^n combinaciones
    for i in range(1, 2**n):
        peso_actual = valor_actual = 0
        combinacion_actual = []
        
        for j in range(n):
            if (i >> j) & 1:  # Si el bit j está activo
                nombre, peso, valor = objetos[j]
                peso_actual += peso
                valor_actual += valor
                combinacion_actual.append(nombre)
        
        if peso_actual <= capacidad and valor_actual > mejor_valor:
            mejor_valor = valor_actual
            mejor_combinacion = combinacion_actual.copy()
    
    peso_total = sum(p for nombre, p, _ in objetos if nombre in mejor_combinacion)
    return mejor_valor, mejor_combinacion, peso_total

valor_fb, combinacion_fb, peso_fb = mochila_fuerza_bruta(objetos, capacidad)
print(f"Fuerza bruta - Valor: {valor_fb}, Objetos: {combinacion_fb}, Peso: {peso_fb}/{capacidad}")

## Desafío 3: Asignación Óptima de Memoria

**Objetivo:** Resolver un problema de asignación de recursos donde se debe maximizar la prioridad total de los procesos asignados, respetando restricciones de memoria y compatibilidad.

**Restricciones:**
- Memoria total limitada
- Algunos procesos no pueden ejecutarse simultáneamente
- Cada proceso tiene memoria requerida y prioridad asociada

**Algoritmos comparados:**
1. **Backtracking:** Poda inteligente del espacio de búsqueda
2. **DFS (Búsqueda en Profundidad):** Exploración sistemática
3. **Fuerza Bruta:** Evaluación exhaustiva de combinaciones

**Aplicación práctica:** Gestión de recursos en sistemas operativos

In [ ]:
# Configuración del problema de memoria
print("=== PROBLEMA DE ASIGNACIÓN ÓPTIMA DE MEMORIA ===")

n_procesos = 5
memoria_total = 50  # MB

# Generar procesos aleatorios
procesos = [(f"P{i+1}", random.randint(5, 25), random.randint(1, 10)) 
            for i in range(n_procesos)]

# Generar restricciones de compatibilidad
restricciones = []
for _ in range(n_procesos // 2):
    p1, p2 = random.sample(range(n_procesos), 2)
    if (p2, p1) not in restricciones:
        restricciones.append((p1, p2))

print(f"Memoria disponible: {memoria_total} MB")
print("\nProcesos:")
for i, (nombre, mem, prio) in enumerate(procesos):
    print(f"  {nombre}: {mem} MB, prioridad {prio}")

print("\nRestricciones:")
for p1, p2 in restricciones:
    print(f"  {procesos[p1][0]} y {procesos[p2][0]} incompatibles")

In [ ]:
# Backtracking para asignación de memoria
def asignacion_backtracking(procesos, memoria_total, restricciones):
    mejor_asignacion, mejor_prioridad = [], 0
    
    def es_valido(asignacion, nuevo_proceso):
        for p1, p2 in restricciones:
            if nuevo_proceso == p1 and p2 in asignacion:
                return False
            if nuevo_proceso == p2 and p1 in asignacion:
                return False
        return True
    
    def backtrack(indice, asignacion, memoria_usada, prioridad_total):
        nonlocal mejor_asignacion, mejor_prioridad
        
        if indice == len(procesos):
            if prioridad_total > mejor_prioridad:
                mejor_asignacion, mejor_prioridad = asignacion.copy(), prioridad_total
            return
        
        # No asignar proceso actual
        backtrack(indice + 1, asignacion, memoria_usada, prioridad_total)
        
        # Asignar proceso actual si es válido
        nombre, mem, prio = procesos[indice]
        if memoria_usada + mem <= memoria_total and es_valido(asignacion, indice):
            asignacion.append(indice)
            backtrack(indice + 1, asignacion, memoria_usada + mem, prioridad_total + prio)
            asignacion.pop()
    
    backtrack(0, [], 0, 0)
    
    nombres = [procesos[i][0] for i in mejor_asignacion]
    memoria_usada = sum(procesos[i][1] for i in mejor_asignacion)
    return nombres, mejor_prioridad, memoria_usada

asign_bt, prio_bt, mem_bt = asignacion_backtracking(procesos, memoria_total, restricciones)
print(f"\nBacktracking: {asign_bt} | Prioridad: {prio_bt} | Memoria: {mem_bt}/{memoria_total} MB")

In [ ]:
# Comparación con otros métodos (simplificado)
def asignacion_fuerza_bruta(procesos, memoria_total, restricciones):
    mejor_asignacion, mejor_prioridad = [], 0
    n = len(procesos)
    
    for mascara in range(1, 2**n):
        asignacion = [i for i in range(n) if (mascara >> i) & 1]
        
        # Verificar memoria
        memoria_total_usada = sum(procesos[i][1] for i in asignacion)
        if memoria_total_usada > memoria_total:
            continue
        
        # Verificar restricciones
        valido = True
        for p1, p2 in restricciones:
            if p1 in asignacion and p2 in asignacion:
                valido = False
                break
        
        if valido:
            prioridad_total = sum(procesos[i][2] for i in asignacion)
            if prioridad_total > mejor_prioridad:
                mejor_asignacion, mejor_prioridad = asignacion, prioridad_total
    
    nombres = [procesos[i][0] for i in mejor_asignacion]
    memoria_usada = sum(procesos[i][1] for i in mejor_asignacion)
    return nombres, mejor_prioridad, memoria_usada

asign_fb, prio_fb, mem_fb = asignacion_fuerza_bruta(procesos, memoria_total, restricciones)
print(f"Fuerza bruta: {asign_fb} | Prioridad: {prio_fb} | Memoria: {mem_fb}/{memoria_total} MB")

## Desafío 4: Resolución de Sudoku

**Objetivo:** Resolver un puzzle de Sudoku 9×9 utilizando el algoritmo de backtracking.

**Reglas del Sudoku:**
- Cada fila debe contener los números 1-9 sin repetición
- Cada columna debe contener los números 1-9 sin repetición  
- Cada subcuadrícula 3×3 debe contener los números 1-9 sin repetición

**Algoritmo backtracking:**
1. Encontrar la próxima casilla vacía
2. Probar números del 1 al 9
3. Verificar si el número es válido según las reglas
4. Si es válido, colocar y continuar recursivamente
5. Si no hay solución, retroceder (backtrack)

**Complejidad:** O(9^(n²)) donde n es el número de casillas vacías

In [ ]:
# Configuración del Sudoku
def crear_tablero():
    return [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

def mostrar_tablero(tablero):
    for i, fila in enumerate(tablero):
        if i % 3 == 0 and i != 0:
            print("------+-------+------")
        
        fila_str = ""
        for j, num in enumerate(fila):
            if j % 3 == 0 and j != 0:
                fila_str += "| "
            fila_str += f"{num if num != 0 else '.'} "
        print(fila_str)

print("=== SUDOKU - BACKTRACKING ===")
tablero = crear_tablero()
print("\nTablero inicial:")
mostrar_tablero(tablero)

In [ ]:
# Funciones de validación
def es_valido(tablero, fila, col, num):
    # Verificar fila
    if num in tablero[fila]:
        return False
    
    # Verificar columna
    if num in [tablero[i][col] for i in range(9)]:
        return False
    
    # Verificar subcuadrícula 3x3
    inicio_fila, inicio_col = 3 * (fila // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if tablero[inicio_fila + i][inicio_col + j] == num:
                return False
    
    return True

def encontrar_vacio(tablero):
    for i in range(9):
        for j in range(9):
            if tablero[i][j] == 0:
                return (i, j)
    return None

In [ ]:
# Resolver Sudoku con backtracking
def resolver_sudoku(tablero):
    vacio = encontrar_vacio(tablero)
    if not vacio:
        return True  # Sudoku resuelto
    
    fila, col = vacio
    
    for num in range(1, 10):
        if es_valido(tablero, fila, col, num):
            tablero[fila][col] = num
            
            if resolver_sudoku(tablero):
                return True
            
            # Backtrack
            tablero[fila][col] = 0
    
    return False

# Resolver y mostrar resultado
tablero_copia = [fila[:] for fila in tablero]

print("\nResolviendo...")
if resolver_sudoku(tablero_copia):
    print("\n✅ Sudoku resuelto:")
    mostrar_tablero(tablero_copia)
else:
    print("❌ No se encontró solución")

## Conclusiones

### Análisis de Rendimiento

| Desafío | Algoritmo | Complejidad | Ventajas | Desventajas |
|---------|-----------|-------------|----------|-------------|
| **Determinantes** | Recursivo | O(n!) | Simple implementación | Muy lento para n > 5 |
| | Iterativo | O(n³) | Eficiente | Más complejo |
| **Mochila** | Backtracking | O(2ⁿ) | Poda inteligente | Exponencial |
| | Fuerza Bruta | O(2ⁿ) | Garantiza óptimo | Sin optimización |
| **Memoria** | Backtracking | O(2ⁿ) | Maneja restricciones | Exponencial |
| | DFS | O(2ⁿ) | Exploración sistemática | Sin poda |
| **Sudoku** | Backtracking | O(9^k) | Solución exacta | Puede ser lento |

### Lecciones Aprendidas

1. **Backtracking** es especialmente útil cuando se pueden aplicar restricciones para podar el espacio de búsqueda
2. **Algoritmos iterativos** suelen ser más eficientes que sus contrapartes recursivas
3. **Fuerza bruta** garantiza encontrar la solución óptima pero no escala bien
4. La **elección del algoritmo** depende del tamaño del problema y los recursos disponibles

### Aplicaciones Prácticas

- **Determinantes:** Álgebra lineal, sistemas de ecuaciones, geometría computacional
- **Mochila:** Optimización de recursos, planificación financiera, logística
- **Asignación de memoria:** Sistemas operativos, gestión de recursos computacionales
- **Sudoku:** Juegos, puzzles lógicos, validación de algoritmos de búsqueda

---

**Fecha de entrega:** 29 de septiembre de 2024  
**Puntos totales:** 25 puntos